In [1]:
from epymorph.geo.spec import DynamicGeoSpec, AttribDef, CentroidDType, Year
from epymorph.geo.adrio.census.adrio_census import Granularity, CensusGeography
from epymorph.data_shape import Shapes
from pathlib import Path
import numpy as np

spec = DynamicGeoSpec(
    attributes=[
        AttribDef('label', np.str_, Shapes.N),
        AttribDef('population', np.int64, Shapes.N),
        AttribDef('population_by_age', np.int64, Shapes.NxA(3)),
        AttribDef('centroid', CentroidDType, Shapes.N),
        AttribDef('geoid', np.str_, Shapes.N),
        AttribDef('dissimilarity_index', np.float64, Shapes.N),
        AttribDef('median_income', np.int64, Shapes.N),
        AttribDef('pop_density_km2', np.float64, Shapes.N)
    ],
    time_period=Year(2015),
    geography=CensusGeography(granularity=Granularity.COUNTY, filter={
        'state': ['04', '08', '49', '35', '32'],
        'county': ['*'],
        'tract': ['*'],
        'block group': ['*']
    }),
    source={
        'label': 'Census:name',
        'population': 'Census',
        'population_by_age': 'Census',
        'centroid': 'Census',
        'geoid': 'Census',
        'dissimilarity_index': 'Census',
        'median_income': 'Census',
        'pop_density_km2': 'Census'
    }
)

json = spec.serialize()
with open(Path('./epymorph/data/geo/us_sw_counties_2015.geo'), mode='w', encoding='utf-8') as f:
    f.write(json)


In [2]:
from epymorph.geo.adrio.census.adrio_census import ADRIOMakerCensus


maker = ADRIOMakerCensus()
if isinstance(spec.geography, CensusGeography) and isinstance(spec.time_period, Year):
    df = maker.fetch_acs5(
         maker.attrib_vars['population'],
         granularity=spec.geography.granularity.value,
         nodes=spec.geography.filter,
         year=spec.time_period.year
)


In [3]:
from epymorph.geo.dynamic import DynamicGeo
from epymorph.geo.adrio import adrio_maker_library

geo = DynamicGeo.from_library(spec, adrio_maker_library)


In [4]:
values = [geo['label'][:5],
          geo['population'][:5],
          geo['population_by_age'][:5],
          geo['centroid'][:5],
          geo['geoid'][:5],
          geo['dissimilarity_index'][:5],
          geo['median_income'][:5],
          geo['pop_density_km2'][:5]]

print(values)


[array(['Apache County, Arizona', 'Cochise County, Arizona',
       'Coconino County, Arizona', 'Gila County, Arizona',
       'Graham County, Arizona'], dtype='<U29'), array([ 72124, 129647, 136701,  53165,  37407]), array([[ 22828,  39037,  10358],
       [ 54262, 110146,  37560],
       [ 88450, 196566,  53745],
       [100071, 223219,  68724],
       [111157, 244490,  73871]]), array([(-109.488846  , 35.39561637), (-109.75114065, 31.87956519),
       (-111.77051224, 35.83872946), (-110.8117366 , 33.79975225),
       (-109.88743768, 32.93271631)],
      dtype=[('longitude', '<f8'), ('latitude', '<f8')]), array([4001, 4003, 4005, 4007, 4009]), array([0.5, 0.5, 0.5, 0.5, 0.5]), array([31757, 45075, 50234, 39751, 45964]), array([2., 8., 3., 4., 3.])]


In [5]:
import os
from epymorph.geo.spec import DynamicGeoSpec, AttribDef, CentroidDType, Year
from epymorph.geo.adrio.census.adrio_census import CensusGeography, Granularity
from epymorph.data_shape import Shapes
from pathlib import Path
import numpy as np

pei = DynamicGeoSpec(
    attributes=[
        AttribDef('label', np.str_, Shapes.N),
        AttribDef('population', np.int64, Shapes.N),
        AttribDef('geoid', np.int64, Shapes.N),
        AttribDef('centroid', CentroidDType, Shapes.N),
        AttribDef('commuters', np.int64, Shapes.NxN)
    ],
    geography=CensusGeography(Granularity.STATE, filter={
        'state': ['12', '13', '45', '37', '51', '24'],
        'county': ['*'],
        'tract': ['*'],
        'block group': ['*']
    }),
    time_period=Year(2015),
    source={
        'label': 'Census:name',
        'population': 'Census',
        'geoid': 'Census',
        'centroid': 'Census',
        'commuters': 'Census'
    }
)

os.makedirs('./scratch/geo', exist_ok=True)

json = pei.serialize()
with open(Path('./scratch/geo/pei.geo'), mode='w', encoding='utf-8') as f:
    f.write(json)


In [6]:
from epymorph.geo.dynamic import DynamicGeo
from epymorph.geo.adrio import adrio_maker_library

geo = DynamicGeo.from_library(pei, adrio_maker_library)


In [7]:
values = [geo['label'][:5],
          geo['population'][:5],
          geo['centroid'][:5],
          geo['geoid'][:5],
          geo['commuters'][:5]]

print(values)


[array(['Florida', 'Georgia', 'Maryland', 'North Carolina',
       'South Carolina'], dtype='<U14'), array([19645772, 10006693,  5930538,  9845333,  4777576]), array([(-82.4974766 , 28.62843614), (-83.44633796, 32.64922284),
       (-76.77245239, 39.03691447), (-79.37236353, 35.54153174),
       (-80.89611244, 33.90795316)],
      dtype=[('longitude', '<f8'), ('latitude', '<f8')]), array([12, 13, 24, 37, 45]), array([[8329464,   12995,    2145,    3659,    1874,    2271],
       [  15309, 4219887,     770,    5502,   20877,    1533],
       [    820,     525, 1987903,     902,     142,  101643],
       [   3142,    6249,    1526, 4251623,   41999,    9564],
       [   1476,   24861,     424,   67325, 1955631,     895]])]


In [8]:
from epymorph.geo.spec import DynamicGeoSpec, AttribDef, CentroidDType, Year
from epymorph.geo.adrio.census.adrio_census import CensusGeography, Granularity
from epymorph.data_shape import Shapes
from pathlib import Path
import numpy as np

maricopa = DynamicGeoSpec(
        attributes=[
            AttribDef('label', np.str_, Shapes.N),
            AttribDef('geoid', np.int64, Shapes.N),
            AttribDef('centroid', CentroidDType, Shapes.N),
            AttribDef('median_income', np.int64, Shapes.N),
            AttribDef('tract_median_income', np.int64, Shapes.N),
            AttribDef('population', np.int64, Shapes.N),
            AttribDef('population_by_age', np.int64, Shapes.NxA(3)),
            AttribDef('population_by_age_x6', np.int64, Shapes.NxA(6)),
            AttribDef('pop_density_km2', np.float64, Shapes.N),
            AttribDef('median_age', np.int64, Shapes.N),
            AttribDef('average_household_size', np.int64, Shapes.N),
            AttribDef('gini_index', np.float64, Shapes.N)
        ],
        geography=CensusGeography(granularity=Granularity.CBG, filter={
            'state': ['04'],
            'county': ['013'],
            'tract': ['*'],
            'block group': ['*']
        }),
        time_period=Year(2015),
        source={
            'label': 'Census:name',
            'population': 'Census',
            'geoid': 'Census',
            'centroid': 'Census',
            'median_income': 'Census',
            'tract_median_income': 'Census',
            'population_by_age': 'Census', 
            'population_by_age_x6': 'Census',
            'pop_density_km2': 'Census',
            'median_age': 'Census', 
            'average_household_size': 'Census',
            'gini_index': 'Census'
        }
    )

json = maricopa.serialize()
with open(Path('./scratch/geo/maricopa_cbg_2019.geo'), mode='w', encoding='utf-8') as f:
    f.write(json)


In [9]:
from epymorph.geo.dynamic import DynamicGeo
from epymorph.geo.adrio import adrio_maker_library

geo = DynamicGeo.from_library(maricopa, adrio_maker_library)


In [10]:
values = [geo['label'][:5],
          geo['population'][:5],
          geo['geoid'][:5],
          geo['centroid'][:5],
          geo['median_income'][:5],
          geo['tract_median_income'][:5],
          geo['population_by_age'][:5],
          geo['population_by_age_x6'][:5],
          geo['pop_density_km2'][:5],
          geo['median_age'][:5],
          geo['average_household_size'][:5],
          geo['gini_index'][:5]]

print(values)


Gini Index cannot be retrieved for block group level, fetching tract level data instead.
[array(['Block Group 1, Census Tract 101.01, Maricopa County, Arizona',
       'Block Group 2, Census Tract 101.01, Maricopa County, Arizona',
       'Block Group 3, Census Tract 101.01, Maricopa County, Arizona',
       'Block Group 1, Census Tract 101.02, Maricopa County, Arizona',
       'Block Group 2, Census Tract 101.02, Maricopa County, Arizona'],
      dtype='<U61'), array([1454, 2127, 1376,  867, 1569]), array([40130101011, 40130101012, 40130101013, 40130101021, 40130101022]), array([(-111.80968903, 33.41876762), (-111.77518314, 33.40445837),
       (-111.93046402, 33.4766662 ), (-112.00301735, 33.57624936),
       (-112.0868316 , 33.49681063)],
      dtype=[('longitude', '<f8'), ('latitude', '<f8')]), array([ 96367, 111440, 108375, 133731, 120491]), array([106818, 106818, 106818, 132822, 132822]), array([[ 354, 1118,   61],
       [ 883, 1935,  892],
       [ 961, 2348, 1855],
       [243

In [11]:
from pathlib import Path
from epymorph.geo.util import convert_to_static_geo

static = convert_to_static_geo(geo)
static.save(Path(f'./scratch/geo/maricopa_cbg_2019.geo.tar'))


In [12]:
from epymorph.geo.static import StaticGeo

static = StaticGeo.load(Path(f'./scratch/geo/maricopa_cbg_2019.geo.tar'))

print(static.values)


{'average_household_size': array([         2,          2,          2, ...,          2,          4,
       -666666666]), 'centroid': array([(-111.80968903, 33.41876762), (-111.77518314, 33.40445837),
       (-111.93046402, 33.4766662 ), ..., (-112.42377283, 33.65641454),
       (-111.98627024, 33.60139296), (-112.207426  , 33.58472109)],
      dtype=[('longitude', '<f8'), ('latitude', '<f8')]), 'geoid': array([40130101011, 40130101012, 40130101013, ..., 40139805001,
       40139806001, 40139807001]), 'gini_index': array([ 4.22600000e-01,  5.01300000e-01,  5.13600000e-01,  4.60800000e-01,
        4.85300000e-01,  4.53900000e-01,  4.60700000e-01,  3.47000000e-01,
        3.67800000e-01,  3.42100000e-01,  3.84700000e-01,  3.70800000e-01,
        3.92800000e-01,  3.05500000e-01,  3.96100000e-01,  3.60200000e-01,
        2.43200000e-01,  3.83300000e-01,  4.06300000e-01,  3.64000000e-01,
        3.74500000e-01,  3.57300000e-01,  3.64300000e-01,  3.59800000e-01,
        3.60600000e-01,  3.3510

In [13]:
from epymorph.geo.cache import fetch
from epymorph.cache import print_geos

fetch('us_sw_counties_2015')
print_geos()


epymorph geo cache contains 1 geos totaling 20.4 kiB (/home/tcoles/.cache/epymorph)
* us_sw_counties_2015 (20.4 kiB)


0

In [14]:
from datetime import date
from functools import partial

from epymorph.data import ipm_library, mm_library, geo_library_dynamic
from epymorph.initializer import single_location
from epymorph.simulation import Simulation, with_fancy_messaging

sim = Simulation(
    geo=geo_library_dynamic['us_sw_counties_2015'](),
    ipm_builder=ipm_library['sirs'](),
    mvm_builder=mm_library['centroids']()
)

with_fancy_messaging(sim)

out = sim.run(
    param={
        'beta': 0.4,
        'gamma': 0.25,  # 1/4
        'xi': 0.0111,  # 1/90
        'phi': 40.0,
        'theta': 0.1,
        'move_control': 0.9,
    },
    start_date=date(2010, 1, 1),
    duration_days=150,
    initializer=partial(single_location, location=0, seed_size=10_000),
)


Running simulation (BasicEngine):
• 2010-01-01 to 2010-05-31 (150 days)
• 158 geo nodes
|####################| 100% 
Runtime: 12.486s
